<a href="https://colab.research.google.com/github/mralamdari/NLP-Word-Embeddings-Beginner/blob/main/NLP_Word_Embeddings_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import collections

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%204/shakespeare.txt

--2022-05-04 14:49:44--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%204/shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306996 (300K) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>] 299.80K  --.-KB/s    in 0.03s   

2022-05-04 14:49:45 (9.05 MB/s) - ‘shakespeare.txt’ saved [306996/306996]



In [3]:
with open('shakespeare.txt', 'r') as f:
  data = f.read()

In [4]:
data = re.sub(r'[,!?:-]', '.', data)
data = nltk.tokenize.word_tokenize(data)
data = [char.lower() for char in data if char.isalpha() or char=='.']

In [5]:
len(data)

60389

In [6]:
frquency_distribution = nltk.FreqDist(word for word in data)

In [7]:
len(frquency_distribution)

5772

In [8]:
frquency_distribution.most_common(40)

[('.', 9086),
 ('the', 1521),
 ('and', 1394),
 ('i', 1252),
 ('to', 1159),
 ('of', 1093),
 ('my', 857),
 ('that', 781),
 ('in', 770),
 ('you', 748),
 ('a', 742),
 ('is', 630),
 ('not', 559),
 ('for', 467),
 ('it', 460),
 ('with', 441),
 ('his', 434),
 ('but', 417),
 ('me', 417),
 ('your', 397),
 ('be', 395),
 ('thy', 359),
 ('this', 355),
 ('he', 353),
 ('so', 352),
 ('thou', 332),
 ('have', 327),
 ('as', 315),
 ('her', 281),
 ('him', 277),
 ('love', 271),
 ('all', 255),
 ('which', 248),
 ('by', 244),
 ('what', 241),
 ('thee', 240),
 ('will', 239),
 ('lord', 225),
 ('no', 220),
 ('shall', 218)]

In [9]:
data_counter = collections.Counter(data)

In [10]:
def word_index_spliter(data):

  word2Ind, Ind2word = {}, {}
  words = sorted(list(set(data)))    

  for index, k in enumerate(words):
    word2Ind[k]  = index
    Ind2word[index] = k
  
  return word2Ind, Ind2word

In [11]:
word2Ind, Ind2word = word_index_spliter(data)

In [15]:
for i in word2Ind.items():
  print(i)
  break

('.', 0)


In [ ]:
rand_ind = np.random.randint(0, len(word2Ind))
rand_word = Ind2word[rand_ind]
print(word2Ind[rand_word], rand_word)

5348 untrimmed


In [ ]:
def weights_initialization(n, v, seed=1):
  
  np.random.seed(seed)
  w1 = np.random.random((n, v))
  w2 = np.random.random((v, n))
  b1 = np.random.random((n, 1))
  b2 = np.random.random((v, 1))

  return w1, w2, b1, b2

In [ ]:
w1, w2, b1, b2 = weights_initialization(4, 8)

In [ ]:
w1

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
        1.46755891e-01, 9.23385948e-02, 1.86260211e-01, 3.45560727e-01],
       [3.96767474e-01, 5.38816734e-01, 4.19194514e-01, 6.85219500e-01,
        2.04452250e-01, 8.78117436e-01, 2.73875932e-02, 6.70467510e-01],
       [4.17304802e-01, 5.58689828e-01, 1.40386939e-01, 1.98101489e-01,
        8.00744569e-01, 9.68261576e-01, 3.13424178e-01, 6.92322616e-01],
       [8.76389152e-01, 8.94606664e-01, 8.50442114e-02, 3.90547832e-02,
        1.69830420e-01, 8.78142503e-01, 9.83468338e-02, 4.21107625e-01]])

In [ ]:
b1

array([[0.04995346],
       [0.53589641],
       [0.66379465],
       [0.51488911]])

In [ ]:
print(w1.shape, b1.shape)

(4, 8) (4, 1)


In [ ]:
print(w2.shape, b2.shape)

(8, 4) (8, 1)


In [ ]:
softmax = lambda z: np.exp(z) / np.sum(np.exp(z), axis=0)

In [ ]:
arr = np.array([[1, 2, 3], [1, 1, 1]])
softmax(arr)

array([[0.5       , 0.73105858, 0.88079708],
       [0.5       , 0.26894142, 0.11920292]])

In [ ]:
def forward_propagation(x, w1, w2, b1, b2):

  hidden_layer = w1.dot(x) + b1
  
  #1
  # hidden_layer[hidden_layer <= 0] = 0
  # activation_layer = hidden_layer
  
  #2
  activation_layer = np.maximum(0, hidden_layer)

  output_layer = w2.dot(activation_layer) + b2
  return output_layer, activation_layer

In [ ]:
x = np.array([[0, 1, 0]]).T

w1, w2, b1, b2 = weights_initialization(2, 3)

out, hid = forward_propagation(x, w1, w2, b1, b2)

In [ ]:
print(out)

[[0.55379268]
 [1.58960774]
 [1.50722933]]


In [ ]:
print(hid)

[[0.92477674]
 [1.02487333]]


In [ ]:
def cost_func(y, pred, batch_size):

  log_probs = np.multiply(np.log(pred), y) + np.multiply(np.log(1-pred), 1-y)
  cost = -1 / batch_size * np.sum(log_probs)
  return cost

In [ ]:
def pack_idx_with_frequency(context_words, word2Ind):
    freq_dict = collections.Counter(context_words)
    idxs = [(word2Ind[word], freq_dict.get(word)) for word in context_words]
    return idxs

In [ ]:
def get_vectors(data, word2Ind, V, C):
    i = C
    while True:
      y = np.zeros(V)
      x = np.zeros(V)
      center_word = data[i]
      y[word2Ind.get(center_word)] = 1
      context_words = data[(i-C): i] + data[(i+1): (i+C+1)]
      num_ctx_words = len(context_words)
      for idx, freq in pack_idx_with_frequency(context_words, word2Ind):
          x[idx] = freq/num_ctx_words
      
      yield x, y
      i += 1
      if i >= len(data):
          print('i is being set to 0')
          i = 0

In [ ]:
def get_batches(data, word2Ind, V, C, batch_size):
    batch_x = []
    batch_y = []
    for x, y in get_vectors(data, word2Ind, V, C):
        while len(batch_x) < batch_size:
            batch_x.append(x)
            batch_y.append(y)
        else:
            yield np.array(batch_x).T, np.array(batch_y).T
            batch = []

In [ ]:
tmp_C = 2
tmp_N = 50
batch_size = 4
tmp_word2Ind, tmp_Ind2word = word_index_spliter(data)
tmp_V = len(word2Ind)

w1, w2, b1, b2 = weights_initialization(tmp_N, tmp_V)

tmp_x, tmp_y = next(get_batches(data, tmp_word2Ind, tmp_V,tmp_C, batch_size))

2


In [ ]:
out, hid = forward_propagation(tmp_x, w1, w2, b1, b2)

In [ ]:
pred = softmax(out)

In [ ]:
cost_func(tmp_y, pred, 4)

12.982478178977658

In [ ]:
def back_propagation(x, y_pred, y, hid, w1, w2, b1, b2, batch_size):

  dist = y_pred-y
  l1 = w2.T.dot(dist)
  l1 = np.maximum(0, l1)

  grad_w1 = (1/batch_size) * l1.dot(x.T)

  grad_w2 = (1/batch_size) * dist.dot(hid.T)

  grad_b1 = np.sum((1/batch_size) * l1.dot(x.T), axis=1, keepdims=True)

  grad_b2 = np.sum((1/batch_size) * dist.dot(hid.T), axis=1, keepdims=True)

  return grad_w1, grad_w2, grad_b1, grad_b2

In [ ]:
grad_W1, grad_W2, grad_b1, grad_b2 = back_propagation(tmp_x, pred, tmp_y, hid, w1, w2, b1, b2, batch_size)

In [ ]:
print(grad_W1.shape)
print(grad_W2.shape)
print(grad_b1.shape)
print(grad_b2.shape)

(50, 5772)
(5772, 50)
(50, 1)
(5772, 1)


In [ ]:
def gradient_descent(data, word2Ind, n, v, num_iterations, alpha=3e-2):
  
  w1, w2, b1, b2 = weights_initialization(n, v, 973)
  batch_size = 128
  iteration = 0
  C = 2

  for x, y in get_batches(data, word2Ind, v, C, batch_size):
    print(x)
    out, hid = forward_propagation(x, w1, w2, b1, b2)

    pred = softmax(out)

    cost = cost_func(y, pred, batch_size)

    grad_w1, grad_w2, grad_b1, grad_b2 = back_propagation(x, pred, y, hid, w1, w2, b1, b2, batch_size)

    w1 -= alpha * grad_w1
    w2 -= alpha * grad_w2 
    b1 -= alpha * grad_b1 
    b2 -= alpha * grad_b2  

    if iteration % 10 == 0:
      print(f'Iteration: {iteration}/{num_iterations}, ==> Cost: {cost}')
    
    print(iteration)
    iteration += 1
    if iteration == num_iterations:
      break

    if iteration % 100 == 0:
      alpha *= 0.66
    

    return w1, w2, b1, b2

In [ ]:
word2Ind, Ind2word = word_index_spliter(data)

N, V, C = 50, len(word2Ind), 2
W1, W2, b1, b2 = gradient_descent(data, word2Ind, N, V, 150)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Iteration: 0/150, ==> Cost: 11.578765890794628
0
